In [ ]:
# YOLOv8 Training Script

from ultralytics import YOLO
import os

# Load a pretrained YOLOv8n variant (custom lightweight model)
model = YOLO('../src/human_detector/models/yolov11n.pt')


# Training with simple parameters
#    - data: path to your data.yaml
#    - epochs, imgsz, project, name: same CLI args
# results = model.train(
#     data    = 'data.yaml',
#     epochs  = 1,
#     imgsz   = 640,
#     project = 'human_robot',
#     name    = 'exp1'
# )


# Train the model with inline arguments
results = model.train(
    data='data.yaml',
    epochs=120,
    patience=20,
    imgsz=416,
    batch=2,
    device=0,
    half=True,
    workers=2,
    cache=False,
    
    # Low-light augmentation settings
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.0
)

# Save trained model path
save_dir = str(results.save_dir)
best_pt = os.path.join(save_dir, 'weights', 'best.pt')
print(f"✅  Trained checkpoint saved to: {best_pt}")


Ultralytics 8.3.141 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce MX550, 1864MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../src/human_detector/models/yolov11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp_lowlight, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=Tru

train: Scanning /home/yogee/Desktop/human_detector_ws/training_data/labels/train.cache... 1324 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1324/1324 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3340.0±2412.2 MB/s, size: 335.7 KB)


val: Scanning /home/yogee/Desktop/human_detector_ws/training_data/labels/val.cache... 331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 331/331 [00:00<?, ?it/s]


Plotting labels to human_robot/exp_lowlight/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 2 dataloader workers
Logging results to human_robot/exp_lowlight
Starting training for 120 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/120     0.275G      1.332      3.111      1.337         11        416:   3%|▎         | 22/662 [00:02<00:59, 10.78it/s]


KeyboardInterrupt: 

# 🧠 YOLOv8 Training Metrics Cheatsheet

## 🔢 Loss Curves

| Metric            | Meaning                                                | Goal / Good Trend                |
|-------------------|--------------------------------------------------------|----------------------------------|
| `train/box_loss`  | Bounding box regression loss (training)                | ↓ Steady decline                 |
| `train/cls_loss`  | Classification loss (training)                         | ↓ Should fall and stabilize     |
| `train/dfl_loss`  | Distribution Focal Loss (for bounding box quality)     | ↓ Smooth drop                   |
| `val/box_loss`    | Box loss on validation set                             | ↓ Best indicator of generalization |
| `val/cls_loss`    | Classification loss on validation                      | ↓ Falling → Good model fit      |
| `val/dfl_loss`    | DFL loss on validation                                 | ↓ Matches train trend = no overfit |

---

## 📈 Detection Metrics

| Metric                | Description                                                 | Goal / Interpretation           |
|------------------------|-------------------------------------------------------------|----------------------------------|
| `metrics/precision(B)` | % of correct detections out of all predictions              | ↑ High = few false positives     |
| `metrics/recall(B)`    | % of actual objects detected                                | ↑ High = few missed detections   |
| `metrics/mAP50(B)`     | Accuracy of boxes at 50% IoU threshold                     | ↑ Main object detection metric   |
| `metrics/mAP50-95(B)`  | Average mAP over IoU 0.5 to 0.95 (stricter)                | ↑ Robust performance measure     |

---

## 🧪 Ideal Curve Patterns

- **Losses**: should steadily decrease.
- **Validation loss** rising while training loss decreases → possible **overfitting**.
- **Precision & Recall**: should increase and plateau near 1.0.
- **mAP**: aim for **>0.90 mAP50** and **>0.80 mAP50-95** for high-quality results.

---

## ⚠️ Tips

- 🧊 Use `half=True` for mixed precision training on supported GPUs.
- 🧠 Watch `val/box_loss` and `mAP50-95` to catch overfitting early.
- 🧪 If metrics plateau early, try increasing `epochs`, `imgsz`, or improving labels.
- 💾 Check `runs/train/.../results.png` for curve visualization.

---
